In [6]:
import os
import random
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from Bio import SeqIO

from transformers import PreTrainedTokenizerFast

from tokenizers import Tokenizer, normalizers, pre_tokenizers
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer
from tokenizers.pre_tokenizers import Whitespace
from tokenizers.processors import TemplateProcessing
from tokenizers.normalizers import Normalizer, Replace, Lowercase

import sentencepiece as spm


LETTER_TO_BASES = {
    "A": "A",
    "B": "CGT",
    "C": "C",
    "D": "AGT",
    "G": "G",
    "H": "ACT",
    "K": "GT",
    "M": "AC",
    "N": "ACGT",
    "R": "AG",
    "S": "CG",
    "T": "T",
    "V": "ACG",
    "W": "AT",
    "Y": "CT",
}

ROOT = 'e:/PlasmidAI' # '/scratch/adibvafa/plasmid-ai/'
DATA_ROOT = f'{ROOT}/data'
DATA_SPLITS = f'{DATA_ROOT}/splits.csv'
DATASET = f'{DATA_ROOT}/plasmids.fasta'
DATASET_TXT = f'{DATA_ROOT}/plasmids.txt'
DATASET_CUTOFF = f'{DATA_ROOT}/plasmids_cutoff.txt'
DATASET_DUMMY =f'{DATA_ROOT}/plasmids_dummy.txt'
DATASET_FINETUNE = f'{DATA_ROOT}/plasmids_finetune.txt'
TOKENIZER = 'dna_bpe_tokenizer_offset.json'

SEED = 42
LEN_CUTOFF = 100_000
VOCAB_SIZE = 4096
NUM_SEQUENCES = 10     #54646
MAX_TOKEN_LENGTH = 32
SPECIAL_TOKENS = ['[UNK]', '[SEP]', '[PAD]', '[CLS]', '[MASK]']

random.seed(SEED)

In [2]:
def read_fasta_to_dataframe(file_path):
    sequences = []
    for record in SeqIO.parse(file_path, "fasta"):
        sequences.append([record.id, str(record.seq), record.description])
    
    dataset = pd.DataFrame(sequences, columns=['ID', 'Sequence', 'Description'])
    return dataset


def preprocess_dna_seqeunce(seq):
    # Clean all whitespaces
    cleaned_seq = seq.replace(' ', '').replace('\n', '').replace('\r', '').replace('\t', '')
    
    # Replace each letter with a random base from LETTER_TO_BASES using random.randint
    replaced_seq = ''
    for letter in cleaned_seq:
        bases = LETTER_TO_BASES[letter]
        random_base = bases[random.randint(0, len(bases) - 1)]
        replaced_seq += random_base
    
    return replaced_seq

In [10]:
# Read plasmid dataset and splits
dataset = read_fasta_to_dataframe(DATASET)
dataset['Sequence'] = dataset['Sequence'].apply(preprocess_dna_seqeunce)

# Select the finetune dataset
data_splits = pd.read_csv(DATA_SPLITS)
finetune_split = data_splits[(data_splits['split'] == 'train') & (data_splits['finetune'] == 1)]
finetune_dataset = dataset[dataset['ID'].isin(finetune_split['id'])]

# Select the cutoff dataset, whose length is less than LEN_CUTOFF
cutoff_dataset = dataset[dataset['Sequence'].apply(lambda x: len(x) < LEN_CUTOFF)]

# Save dataset to txt for tokenizer
dataset['Sequence'].to_csv(DATASET_TXT, index=False, header=False)
cutoff_dataset['Sequence'].to_csv(DATASET_CUTOFF, index=False, header=False)
finetune_dataset['Sequence'].to_csv(DATASET_FINETUNE, index=False, header=False)
dataset['Sequence'].iloc[:NUM_SEQUENCES].to_csv(DATASET_DUMMY, index=False, header=False)

In [7]:
# Load the SentencePiece tokenizer
tokenizer = PreTrainedTokenizerFast(
    tokenizer_file=TOKENIZER
)

In [4]:
# Test the tokenizer on a sample DNA sequence
sequence = "ATTCTGCGGTTCCCCCTGGAAGACCTACGCAAGTTGGGCCAGCTCAGAGGTGGAATCAACGAAGGCGAGC"
encoded = tokenizer(sequence)
print("Encoded sequence:", encoded)

# Decode the tokens back to the original sequence
decoded_sequence = tokenizer.decode(encoded['input_ids'])
print("Decoded sequence:", decoded_sequence.upper())

# Anlyze the vocabulary
print(f'Alphabete: {set(''.join(list(sorted(tokenizer.vocab.keys())[::-1])[5:]))}')

Encoded sequence: {'input_ids': [3, 5, 502, 612, 43, 162, 86, 40, 64, 23, 680, 245, 1444, 26, 260, 889, 103, 1], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
Decoded sequence: [CLS] A TTCTGC GGTTCC CCC TGGAA GACC TAC GCAA GTT GGGCCA GCTCA GAGGTG GAA TCAAC GAAGGC GAGC [SEP]
Alphabete: {'G', 'A', 'T', 'C'}


In [5]:
sorted(tokenizer.vocab.keys())

['A',
 'AA',
 'AAA',
 'AAAA',
 'AAAAAA',
 'AAAAAAA',
 'AAAAAAAA',
 'AAAAAAC',
 'AAAAAACA',
 'AAAAAACC',
 'AAAACA',
 'AAAACC',
 'AAAACCA',
 'AAAATAA',
 'AAAATCA',
 'AAAC',
 'AAACA',
 'AAACAA',
 'AAACAAA',
 'AAACAAAA',
 'AAACAAC',
 'AAACAC',
 'AAACACA',
 'AAACC',
 'AAACCA',
 'AAACCC',
 'AAACGAC',
 'AAACTCTGC',
 'AAAGAA',
 'AAAGAAA',
 'AAAGAC',
 'AAAGCA',
 'AAAGCC',
 'AAAGGC',
 'AAAGTC',
 'AAAGTG',
 'AAATA',
 'AAATAA',
 'AAATAAA',
 'AAATAC',
 'AAATC',
 'AAATCA',
 'AAATCC',
 'AAATGC',
 'AAATT',
 'AAATTA',
 'AAATTAA',
 'AAC',
 'AACA',
 'AACAA',
 'AACAAA',
 'AACAAAA',
 'AACAAAAA',
 'AACAAAAC',
 'AACAAAC',
 'AACAAC',
 'AACAACA',
 'AACAACC',
 'AACACA',
 'AACC',
 'AACGAA',
 'AACGAC',
 'AACGCC',
 'AACGGA',
 'AACGGC',
 'AACGTC',
 'AACGTG',
 'AACGTGA',
 'AACGTT',
 'AACTAA',
 'AACTAC',
 'AACTCC',
 'AACTGA',
 'AACTGC',
 'AACTGGC',
 'AACTT',
 'AACTTC',
 'AACTTTT',
 'C',
 'CA',
 'CAC',
 'CACA',
 'CACAA',
 'CACAAA',
 'CACAAAA',
 'CACAAAC',
 'CACAAC',
 'CACAACC',
 'CACACA',
 'CACC',
 'CACCAA',
 'CACCAAA

In [ ]:
#################### DEAD ZONE - DO NOT ENTER ####################
# dataset['Sequence'].transform(len).plot(kind='hist', bins=int(1e3))
# plt.xlim(0, 1e6)
# plt.show()

# # Train the SentencePiece model with spm

# with tqdm(total=NUM_SEQUENCES, desc="Training Tokenizer...", unit="sequences") as pbar:

#     spm.SentencePieceTrainer.train(
#         input=DATASET_TXT,                      # Input file containing DNA sequences
#         model_prefix=TOKENIZER,                 # Prefix for the output model files
#         vocab_size=VOCAB_SIZE,                  # Vocabulary size
#         model_type='bpe',                       # Model type (BPE)
#         pad_id=0,                               # ID for padding token
#         unk_id=1,                               # ID for unknown token
#         bos_id=2,                               # ID for beginning-of-sequence token
#         eos_id=3,                               # ID for end-of-sequence token
#         user_defined_symbols=SPECIAL_TOKENS,    # Special tokens
#         character_coverage=1.0,                 # Ensure full coverage of the input characters
#         input_sentence_size=NUM_SEQUENCES,      # Limit the number of sentences for training for efficiency
#         shuffle_input_sentence=True             # Shuffle the input sentences to improve training
#     )
#     pbar.update(NUM_SEQUENCES)

# # Load the SentencePiece model
# sp = spm.SentencePieceProcessor()
# sp.load(f'{TOKENIZER}.model')

# # Tokenize a sentence
# sequence = "ATTCTGCGGTTCCCCCTGGAAGACCTACGCAAGTTGGGCCAGCTCAGAGGTGGAATCAACGAAGGCGAGC"
# tokens = sp.encode_as_pieces(sequence)
# print("Tokens:", tokens)

# # Convert tokens back to text
# decoded_text = sp.decode_pieces(tokens)
# print("Decoded text:", decoded_text)